In [7]:
#Create the Options first


import google.auth
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
#from apache_beam.utils.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions

options = PipelineOptions()

#Setting the Options Programmatically
options = PipelineOptions(flags=[])

#set the project to the default project in your current Google Cloud Environment
_, options.view_as(GoogleCloudOptions).project=google.auth.default()

# Sets the pipeline mode to streaming, so we can stream the data from PubSub.
options.view_as(StandardOptions).streaming = True

#Set the Google Cloud Region in which Cloud Dataflow run
options.view_as(GoogleCloudOptions).region='us-west1'

#Cloud Storage Location
dataflow_gcs_location='gs://gcp-dataeng-demos-soumya/dataflow'

#Dataflow Staging Location. This location is used to stage the Dataflow Pipeline and SDK binary
options.view_as(GoogleCloudOptions).staging_location = '{}/staging'.format(dataflow_gcs_location)
#Dataflow Temp Location. This location is used to store temporary files or intermediate results before finally outputting
options.view_as(GoogleCloudOptions).temp_location = '{}/temp'.format(dataflow_gcs_location)
#The directory to store the output files of the job
output_gcs_location = '{}/output'.format(dataflow_gcs_location)

In [25]:
import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib
import json





with beam.Pipeline(InteractiveRunner(),options=options) as p:
    stage1 = p|"Input" >> beam.io.ReadFromText('gs://gcp-dataeng-demos-soumya/dataflow/json-timestamp.json')
    stage2 = stage1| "Make Dictionary">>beam.FlatMap(lambda x: json.loads(x)) #Use FlatMap instead of Map to unpack the list of dictionary items
    stage3 = stage2| "Make Timestamped">>beam.Map(lambda x: beam.window.TimestampedValue(x, x['Time']))
    stage4 = stage3| "window" >> beam.WindowInto(beam.window.FixedWindows(22))
    stage5 = stage4| "Summarisation">>beam.Count().without_defaults()
    
ib.show_graph(p)    
ib.show(stage1, include_window_info=True)
ib.show(stage2, include_window_info=True)
ib.show(stage3, include_window_info=True)
ib.show(stage4, include_window_info=True)
ib.show(stage5, include_window_info=True)

AttributeError: module 'apache_beam' has no attribute 'Count'